<a href="https://colab.research.google.com/github/Murcicrum/Identificador-de-medio/blob/main/identificador_de_medios_limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cargado de datos crudos

In [ ]:
import os

def get_tweetstext(path_jsons)-> dict: 
    '''
    Dado el path a una carpeta que contenga archivos USUARIOX_tweets.json
    Devuelve diccionario con USUARIOX como claves 
    y una lista con los textos de sus tweets como valor
    '''
    tweets_text = {}
    
    for filename in os.listdir( path_jsons ):
      texts = []
      
      if filename[-5:] == '.json':
          json_data = json.load( open(path_jsons+filename) )
      else: continue
      
      for element in json_data['data']:
          text = element['text']
          texts.append(text)
          
      tweets_text[filename.replace('_tweets.json', '')] = texts
        
    return tweets_text


def print_tweets(tweets_dict, key=None):
    '''
    Dado un diccionario imprime ordenadamente su contenido
    Si una key del mismo es especificada, imprime sólo el valor de esa key
    Si no se especifica key se imprimen recursivamente todas 
    '''
    if key:
        print('\n\n***', key)
        for value in tweets_dict[key]:
           print('->', value)
    else:
        for key in tweets_dict:
            print_tweets(tweets_dict, key)

#Agregar acá el path a una carpeta con .json's
path = ''
tweets = get_tweetstext(path)

##Scrapeo
Para los medios Página12 e IPNoticias, cuyos tweets no contienen el titular de la nota sino un fragmento de la misma, extraigo los links a las notas completas y luego hago scraping en ellos para extraer el titular.

####Obtener links
Cada tweet finaliza con el link a la nota completa, pero en algunos casos este es precedido por otro link a algun sitio adicional.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from time import sleep
from random import random

def get_links(tweets_list) -> list:
    links_list = []
    for tweet in tweets_list:
        i_link = tweet.rfind('http')
         
        if i_link != -1:                              #i.e. si encontró un link, ya que si el tweet no tiene link rfind devuelve -1
            links_list.append( tweet[i_link:] )
        
    return links_list

Pero estos links están comprimidos por twitter EJM, no son explicitamente links a los medios EJM. Con lo cual debo hacer un scraper intermedio que visite cada uno de los links comprimidos y extraiga de allí los links completos y reales de los medios.

In [ ]:
def get_reallinks(twitter_links, verbose=False) -> list:
    '''
    Dada una lista de links comprimidos por twitter
    Devuelve los links reales/descomprimidos
    '''
    real_links = []
    
    for i, link in enumerate(twitter_links):
        if verbose: print('Procesando link ',i, '\t', link)
        try:
            response = requests.get(link)
            real_links.append(response.url)
        except: print('Falló lectura del link', i, '\t', link)
    
    return real_links

####Extracción de los titulares
Defino la funciones que hacen el scraping propiamente dicho. Visitan cada link obtenido con get_link y lo recorren, de cierta forma especifica en cada medio, para extraer el titular.

In [ ]:
def get_ipn_titles(ipn_links, verbose=False) -> list:
    '''
    Dada una lista de links de artículos de IPNoticias
    Devulve los titulares de dichos artículos
    '''
    ipn_titles = []

    for i,link in enumerate(ipn_links):
        if verbose: print('Link ', i, '\t',link)

#Me quedo con los links que remiten a notas escritas, descartando videos y directos
        if link[:24]=='https://ip.digital/nota/':                               
            response = requests.get(link)
            soup = bs(response.content)
            try:
                title = soup.find_all('title')[0].text
                i = title.find(' |')
                ipn_titles.append( title[:i] )
            except: print('Falló lectura del link ', i)
        
        elif verbose: print('El link no pertenece a una nota de IPNoticias.\n')
        
        sleep(10*random())
    return ipn_titles                


def get_p12_titles(p12_links, verbose=False) -> list:
    '''
    Dada una lista de links de artículos de Página12
    Devulve los titulares de dichos artículos
    '''
    p12_titles = []

    for i, link in enumerate(p12_links):
        
        if verbose: print('Link ', i, '\t',link)
        
        if link[:28]=='https://www.pagina12.com.ar/':
          response = requests.get(link)
          soup = bs(response.content)
          try:
              title = soup.find_all('title')[0].text
              i = title.find(' |')
              p12_titles.append( title[:i] )
          except: print('Falló lectura del link ', i)
        
        elif verbose : print('El link no pertenece a una nota de Página12.\n')

        sleep(10*random())
    
    return p12_titles

#Limpieza
Elimino de los tweets todo aquello que no sea el titular de la nota.
Dado que cada medio adorna sus tweets de diferentes formas, hay una función para cada medio (excepto Página12 e IPNoticias que salen limpios del scrapeo).
Recordemos que en todos los casos de todos los medios, el tweet es finalizado con un link, encontrarlo y eliminarlo es el primer paso en común para todos los medios.

In [ ]:
import re
import string

letters = string.ascii_letters
symbols = string.punctuation + '¿¡'

def delete_links(tweets_list):
    #Elimina el link que esté al final del tweet
    #Si el tweet tiene 2 links, hay que pasarlo por esta función 2 veces.
    tweets = tweets_list.copy()
    for n, tweet in enumerate(tweets):
        i_link = tweet.rfind('http')
        if i_link != -1:
            tweets[n] = tweet[:i_link]
            
    return tweets

Una vez que al contenido de los tweets de todos los medios se le eliminaron los links del final, se pasan los tweets de cada medio por una función particular que los limpia de la forma que corresponda.

Por lo general la 'suciedad' en los tweets que queremos eliminar viene en forma de emojis, de hashtags no integrados en el texto, de menciones a las cuentas de los actores de la noticia o a sus redactores,o caracteres especiales utilizados para darle algún formato al tweet. Puede ser que incluso algunos medios publiquen un tweet que no referencie en absoluto a una nota periodística y sea sólo publicidad, una alerta de tránsito o clima, o un retweet a otra cuenta; en dichos casos optamos por descartar el tweet.


####TN

In [ ]:
def clean_tn(tn_titles):
    titles = delete_links( tn_titles )
    
    #Algunos titulares bajan 2 lineas y ponen info de le autore
    for n, title in enumerate(titles):
        i_newline = title.find('\n\n')
        if i_newline -1:
            titles[n] = title[:i_newline]
    
    return titles

####LN+

In [ ]:
def clean_lnmas(lnmas_titles):    
    #Algunos titulares tienen doble link
    titles = delete_links( lnmas_titles )
    titles = delete_links(titles)
    
    #Los que contienen esos hashtags son publicidad de los programas del canal
    titles = [ tit for tit in titles if '#MásMañana' not in tit
                                     if  '#MasData'  not in tit 
                                     if    '#HOY'    not in tit ]
    return titles

####Clarín

In [ ]:
def clean_clarin(clarin_titles):    
    titles = delete_links( clarin_titles )
    
    #Algunos titulares inician con un emoji (que no es letra ni simbolo))
    titles = [ title[1:] if title[0] not in (letters+symbols) 
                         else title
                         for title in titles] 
                         
    #Algunos ponen el link del final en una oración que inicia con 'Más:'    
    for n,title in enumerate(titles):
        i_mas = title.rfind('. Más:')
        if i_mas != -1:
            titles[n] = title[:i_mas]
            
    return titles

#Armado de dataset

Una vez conseguidos los titulares de las noticias limpios procedimos a preprocesarlos con un lematizador. Buscamos cuales eran los lematizadores disponibles en idioma español y luego de hacer algunas pruebas con cada uno optamos por utilizar NLTK.

Una vez tenidos entonces todos los titulares lematizados y sin stopwords, el armado del dataset consistió en guardar cada titular junto con el medio del que provino en un archivo .csv.